In [2]:
import pandas as pd
import urllib.parse
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_main = pd.read_csv("raw_data.csv", low_memory=False)
df = df_main[["id","location","Zip","Type","Subtype",
    "Price","Transaction Type","Bedrooms","Living area","Kitchen type",
    "Furnished","How many fireplaces?","Terrace","Terrace surface",
    "Garden","Garden surface","Surface of the plot","Number of frontages",
    "Swimming pool","Building condition","Primary energy consumption"]]
df = df.set_index("id")

In [4]:
df = df.drop(df[df["Type"]=="house group"].index)
df = df.drop(df[df["Type"]=="apartment group"].index)

In [5]:
df = df.rename(columns={
    'location' :'Locality',
    'Transaction Type' : 'Type of sale',
    'Type' :'Type of property',
    'Subtype' : 'Subtype of property',
    'Number of frontages': 'Number of facades',
    'Bedrooms':'Number of rooms',
    'Surface of the plot' :'Surface of the land',
    'Kitchen type' : 'Fully equipped kitchen',
    'How many fireplaces?' : 'Open fire'
})
 
#put the columns in a specific order'
df = df.reindex(columns=['Locality',"Zip", 'Type of property', 'Subtype of property',
                         'Type of sale', 'Price',
                         'Building condition','Building Cond. values',
                         'Number of facades', 'Number of rooms', 
                         'Living area',
                         'Furnished','Fully equipped kitchen','Kitchen values',
                         'Surface of the land',
                         'Primary energy consumption','Energy_classes' ,
                         'Terrace', 'Terrace surface','Garden','Garden surface',
                         'Open fire', 'Swimming pool'])


In [6]:
def clean_and_convert(column):
    column = column.apply(lambda x: re.sub('\D+', '', str(x)))
    column = column.replace('', np.nan)
    return column

df['Living area'] = clean_and_convert(df['Living area'])
df['Terrace surface'] = clean_and_convert(df['Terrace surface'])
df['Garden surface'] = clean_and_convert(df['Garden surface'])
df['Surface of the land'] = clean_and_convert(df['Surface of the land'])
df['Primary energy consumption'] = clean_and_convert(df['Primary energy consumption'])

df['Locality'] = df['Locality'].apply(urllib.parse.unquote)

In [7]:
conditions = [
    (df['Garden']== "Yes"),
    (df["Garden"].isna()) & (df["Garden surface"].isna()),
    (df["Garden surface"].notna())
    ]
values = [1, 0, 1]
df['Garden'] = np.select(conditions, values)

df.loc[(df["Garden"] == 0 ) & (df["Garden surface"].isna()), 'Garden surface'] = 0

In [8]:
conditions = [
    (df['Terrace']== "Yes"),
    (df["Terrace"].isna()) & (df["Terrace surface"].isna()),
    (df["Terrace surface"].notna())
    ]
values = [1, 0, 1]
df['Terrace'] = np.select(conditions, values)

df.loc[(df["Terrace"] == 0 ) & (df["Terrace surface"].isna()), 'Terrace surface'] = 0

In [9]:
def nan_replacement(column):
    column = column.replace("Yes",1)
    column = column.replace("No",0)
    column = column.replace('', np.nan).fillna(0)
    return column

df['Furnished'] = nan_replacement(df['Furnished'])
df['Swimming pool'] = nan_replacement(df['Swimming pool'])
df['Open fire'] = nan_replacement(df['Open fire'])

In [10]:
df["Price"] = df["Price"].astype(int)
df = df.astype({"Price":"int",
                "Number of rooms":"float",
                "Living area":"float",
                "Terrace surface":"float",
                "Garden surface":"float",
                "Surface of the land":"float",
                "Number of facades":"float",
                "Primary energy consumption":"float"})

In [11]:
# Mapping dictionary for replacing values in the "kitchen" column
kitchen_mapping = {
    # np.nan: -1,
    'Not installed': 0,
    'Installed': 1,
    'Semi equipped': 2,
    'Hyper equipped': 3,
    'USA uninstalled' :0,
    'USA installed': 1,
    'USA semi equipped': 2,
    'USA hyper equipped' :3
}
# Replace values in the "Kitchen type" column with corresponding numbers and create a new column called "Kitchen values"
df['Kitchen values'] = df['Fully equipped kitchen'].map(kitchen_mapping).fillna(df['Fully equipped kitchen'])

In [12]:
building_cond_mapping = {
    # np.nan: -1,
    'To restore': 0,
    'To be done up': 2,
    'Just renovated': 3,
    'To renovate': 1,
    'Good': 3,
    'As new' :4
}

df['Building Cond. values'] = df['Building condition'].map(building_cond_mapping).fillna(df['Building condition'])

In [13]:
df = df.drop(df[df["Living area"].isna()].index)

In [14]:
#Missing values fillied with 1
df["Primary energy consumption"] = np.where((df["Primary energy consumption"] != int) & (df["Primary energy consumption"] == ""), np.nan, df["Primary energy consumption"])
# df["Primary energy consumption"] = df["Primary energy consumption"].replace("",np.nan).fillna(-1).astype(int)

#New column with energy classes 
conditions = [
    # (df['Primary energy consumption']==-1),
    (df['Primary energy consumption']>=1)&(df['Primary energy consumption']<100),
    (df['Primary energy consumption']>=100)&(df['Primary energy consumption']<200),
    (df['Primary energy consumption']>=200)&(df['Primary energy consumption']<300),
    (df['Primary energy consumption']>=300)&(df['Primary energy consumption']<400),
    (df['Primary energy consumption']>=400)&(df['Primary energy consumption']<500),
    (df['Primary energy consumption']>=500)&(df['Primary energy consumption']<600),
    (df['Primary energy consumption']>=600)
]

values = [7, 6, 5, 4, 3, 2, 1]

df['Energy_classes'] = np.select(conditions, values)

In [15]:
#added a column for ease of use 
#and later grouped all information by region
def get_province(zip_code):
    if 1000 <= zip_code <= 1299:
        return 'Brussels Capital Region'
    elif 1300 <= zip_code <= 1499:
        return 'Walloon Brabant'
    elif 1500 <= zip_code <= 1999 or 3000 <= zip_code <= 3499:
        return 'Flemish Brabant'
    elif 2000 <= zip_code <= 2999:
        return 'Antwerp'
    elif 3500 <= zip_code <= 3999:
        return 'Limburg'
    elif 4000 <= zip_code <= 4999:
        return 'Liège'
    elif 5000 <= zip_code <= 5999:
        return 'Namur'
    elif 6000 <= zip_code <= 6599 or 7000 <= zip_code <= 7999:
        return 'Hainaut'
    elif 6600 <= zip_code <= 6999:
        return 'Luxembourg'
    elif 8000 <= zip_code <= 8999:
        return 'West Flanders'
    elif 9000 <= zip_code <= 9999:
        return 'East Flanders'
    else:
        return 'Unknown'
        
df['Province'] = df['Zip'].apply(get_province)

In [16]:
df['Price of square meter'] = df['Price'] / df['Living area']
df['Price of square meter'] = df['Price of square meter'].round(2)

In [17]:
df = df.reindex(columns=['Locality',"Zip",'Province', 
                         'Type of property', 'Subtype of property',
                         'Type of sale', 'Price','Price of square meter',
                         'Building condition','Building Cond. values',
                         'Number of facades', 'Number of rooms', 
                         'Living area',
                         'Furnished','Fully equipped kitchen','Kitchen values',
                         'Surface of the land',
                         'Primary energy consumption','Energy_classes' ,
                         'Terrace', 'Terrace surface','Garden','Garden surface',
                         'Open fire', 'Swimming pool'])

In [18]:
print(f"The DataFrame has {df.shape[0]} rows and {df.shape[1]} columns.")

The DataFrame has 18051 rows and 25 columns.


In [19]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']

df_houses = df[df['Type of property'] == 'house']
df_apartments = df[df['Type of property'] == 'apartment']

apartdf = df_apartments.select_dtypes(include=numerics)
noIdApartDf = apartdf.loc[:, ~apartdf.columns.isin(['id'])]

housedf = df_houses.select_dtypes(include=numerics)
noIdHouseDf = housedf.loc[:, ~housedf.columns.isin(['id', 'Zip'])]

In [20]:
print("House DataFrame shape (before): ",housedf.shape)
print("House data min (with outliers): ",housedf['Price'].min())
print("House data max (with outliers): ",housedf['Price'].max())

from typing import List
# Remove outliers
def remove_outliers(df: pd.DataFrame, columns: List[str], n_std: int) -> pd.DataFrame:
    for col in columns:
        print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

new_housedf = remove_outliers(housedf, ['Price'], 4)

print("_"*30)
print("House DataFrame shape (after removing): ",new_housedf.shape)
print("House data min (without outliers): ", new_housedf['Price'].min())
print("House data max (without outliers): ", new_housedf['Price'].max())

House DataFrame shape (before):  (9533, 18)
House data min (with outliers):  2500
House data max (with outliers):  11000000
Working on column: Price
______________________________
House DataFrame shape (after removing):  (9421, 18)
House data min (without outliers):  2500
House data max (without outliers):  3100000


In [21]:
new_housedf.to_csv("clean_house_data.csv")

In [22]:
print("Apartment DataFrame shape (before): ",apartdf.shape)
print("Apartment data min (with outliers): ",apartdf['Price'].min())
print("Apartment data max (with outliers): ",apartdf['Price'].max())

new_apartdf = remove_outliers(apartdf, ['Price'], 3)

print("_"*30)
print("House DataFrame shape (after removing): ",new_apartdf.shape)
print("Apartment data min (without outliers): ",new_apartdf['Price'].min())
print("Apartment data max (without outliers): ",new_apartdf['Price'].max())

Apartment DataFrame shape (before):  (8518, 18)
Apartment data min (with outliers):  10000
Apartment data max (with outliers):  8100000
Working on column: Price
______________________________
House DataFrame shape (after removing):  (8314, 18)
Apartment data min (without outliers):  10000
Apartment data max (without outliers):  1850000


In [ ]:
new_apartdf.to_csv("clean_apt_data.csv")